<a href="https://colab.research.google.com/github/simulate111/Deep-Learning-in-Human-Language-Technology/blob/main/masked_language_modelling_Ex9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BERT for masked language modelling


In [1]:
!pip3 -q install datasets transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.3 MB/s eta 0:00:00


In [2]:
import transformers
import datasets
import torch

# BERT: masked language modelling output

* But BERT is trained to predict masked words, let's try!

In [3]:
tokenizer = transformers.AutoTokenizer.from_pretrained("bert-base-cased") #you can also use the trusty "TurkuNLP/bert-base-finnish-cased-v1"
bert = transformers.AutoModelForPreTraining.from_pretrained("bert-base-cased")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

In [4]:
# Tell the model it is not really being trained (disables dropout for example)
# I do not think this is needed but am playing it safe, the docs say it is put to eval mode upon load: https://huggingface.co/docs/transformers/main_classes/model#transformers.PreTrainedModel.from_pretrained.config
bert = bert.eval()

...now let's see how well this works for the masked word prediction...
* we need to find the most likely predicted words
* which can be achieved by arg-sorting the predictions and picking top N words
* this is easy and we have done this kind of stuff before
* now let's try straight in torch without a roundtrip to numpy

In [5]:
texts = ["Dogs like to [MASK] cats. They are cute."]

t = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")
bert_out = bert(**t)
print(bert_out["prediction_logits"].shape)
top20 = torch.argsort(bert_out["prediction_logits"], dim=2, descending=True)[:,:,:20]
print(top20.shape)

print("Guesses:",tokenizer.decode(top20[0,4]))

torch.Size([1, 12, 28996])
torch.Size([1, 12, 20])
Guesses: have eat chase see pet keep play watch get scare be hunt ride like visit kill feed fight lick catch


In [6]:
print(t)
print(tokenizer.mask_token_id)

{'input_ids': tensor([[  101, 16406,  1176,  1106,   103, 11771,   119,  1220,  1132, 10509,
           119,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
103


# TASKS

As an exercise, you can try to solve the following:

1. How good is BERT at the masked language modelling (MLM) task? Feed random texts e.g. from the IMDB dataset, mask a random token at a time, and check: did BERT predict it correctly?
2. If you did (1), can you answer did BERT predict it correctly in top-5?
3. Try can you do better. Make yourself a program which picks random texts from one of the datasets we used in this course and produces two files: one with segments of texts with one [MASK] and one with the correct answers. Then try to guess the words without looking at the latter file and then compare your answers with the correct ones. How well did you do?


# General notes:

* Some had problems (or really bad reported performance) because of failing to select predictions for the correct token
* In the example code, the index of the masked token was hard-coded: **top20[0,4]**
* A typical error was to manually select the index based on the original text, not the tokenized text
  - _Dogs like to [MASK] cats. They are cute._ --> fourth token, so index=3
  - _[CLS] Dogs like to [MASK] cats . They are cute . [SEP]_ --> fifth token, so index=4
  - Note that tokenizer can also split tokens into subwords, so cannot assume just +1 for inserted CLS!
  - _[CLS] D ##O ##GS like to [MASK] cats . They are cute . [SEP]_ --> seventh token, so index=6




In [7]:
print(dir(tokenizer))
text = "Dogs like to [MASK] cats. They are cute."
tok = tokenizer(texts[0])
print(tok)
detok = tokenizer.convert_ids_to_tokens(tok["input_ids"])
print(detok)
print(detok[4])

['SPECIAL_TOKENS_ATTRIBUTES', '__annotations__', '__call__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_add_tokens', '_additional_special_tokens', '_auto_class', '_batch_encode_plus', '_bos_token', '_call_one', '_cls_token', '_compile_jinja_template', '_convert_encoding', '_convert_id_to_token', '_convert_token_to_id_with_added_voc', '_create_repo', '_decode', '_decode_use_source_tokenizer', '_encode_plus', '_eos_token', '_eventual_warn_about_too_long_sequence', '_eventually_correct_t5_max_length', '_from_pretrained', '_get_files_timestamps', '_get_padding_truncation_strategies', '_in_target_context_manager', '_mask_token', '_pad', '_pad_token', '_pad_token_type_id', '_processo

In [8]:
text = "DOGS LIKE TO [MASK] CATS. THEY ARE CUTE."
tok = tokenizer(text)
print(tok)
detok = tokenizer.convert_ids_to_tokens(tok["input_ids"])
print(detok)
print(detok[8])

{'input_ids': [101, 141, 2346, 13472, 149, 2240, 22441, 16972, 103, 8784, 11365, 119, 7462, 3663, 22133, 2036, 140, 16830, 2036, 119, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
['[CLS]', 'D', '##O', '##GS', 'L', '##I', '##KE', 'TO', '[MASK]', 'CA', '##TS', '.', 'THE', '##Y', 'AR', '##E', 'C', '##UT', '##E', '.', '[SEP]']
[MASK]


# Genaralize to mask a random token and get predictions for it

In [9]:
#Get some data
dataset = datasets.load_dataset("imdb", split="train") # we need only training section
dataset = dataset.shuffle()
dataset = dataset.select(range(100)) # downsample
print(dataset)
print(dataset[0])

README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'label'],
    num_rows: 100
})
{'text': "The film as entertainment is very good and Jimmy Stewart is excellent as Chip Hardesty, with well done co-starring turns by Vera Miles and Murray Hamilton. But the film, directed by legendary director Mervyn Leroy, was constantly vetted and script approval as well as every aspect of the film, down to clothing, was closely watched and controlled by J. Edgar Hoover. Not that J. Edgar Hoover didn't have something to be proud of. His management of the bureau from 1924 to his death crated on of the finest investigative services in the world. But by 1959 Hoover was already beginning to worry about being forced out and had already started to collect dossiers on powerful people to make sure and protect his little kingdom. And he was determined to make sure that no motion picture showed even a single wart about the bureau. The films shows only continued successes and glosses over the failures which occurred, and the burea

In [10]:
# truncate review texts to make it easier to spot things, this step is not necessary!
# Note that this can make the task easier / more difficult compared to full reviews

def truncate_review(example):
  text = example["text"]
  tokens = text.split() # naive tokenization, split on whitespace
  tokens = tokens[:30] # let's keep first 30 tokens
  short_text = " ".join(tokens) # convert back to string
  return {"text": short_text}

dataset = dataset.map(truncate_review)
print(dataset)
print(dataset[0])

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'label'],
    num_rows: 100
})
{'text': 'The film as entertainment is very good and Jimmy Stewart is excellent as Chip Hardesty, with well done co-starring turns by Vera Miles and Murray Hamilton. But the film, directed', 'label': 1}


In [11]:
# Let's test how masking of a random token works

import random

#help(tokenizer.decode)

text = dataset[0]["text"]

t = tokenizer(text) # prepare normal tokenized input
print("input_ids:", t["input_ids"])
print("tokens:", tokenizer.decode(t["input_ids"]))

random_idx = random.randint(1, len(t["input_ids"])-2) # sample random token, -2 so that we do not mask the last token which is [SEP]
print("random index:", random_idx, tokenizer.decode(t["input_ids"][random_idx]))

t["input_ids"][random_idx] = tokenizer.mask_token_id # mask the token in input
print("masked tokens:", tokenizer.decode(t["input_ids"]))




input_ids: [101, 1109, 1273, 1112, 5936, 1110, 1304, 1363, 1105, 4479, 5272, 1110, 6548, 1112, 20379, 9322, 2556, 1183, 117, 1114, 1218, 1694, 1884, 118, 3937, 3587, 1118, 13050, 7726, 1105, 5593, 4436, 119, 1252, 1103, 1273, 117, 2002, 102]
tokens: [CLS] The film as entertainment is very good and Jimmy Stewart is excellent as Chip Hardesty, with well done co - starring turns by Vera Miles and Murray Hamilton. But the film, directed [SEP]
random index: 6 very
masked tokens: [CLS] The film as entertainment is [MASK] good and Jimmy Stewart is excellent as Chip Hardesty, with well done co - starring turns by Vera Miles and Murray Hamilton. But the film, directed [SEP]


In [12]:
# define function to tokenize and mask
def tokenize_and_mask_random(text):
  t = tokenizer(text, return_tensors="pt")
  random_token_idx = random.randint(1, t["input_ids"].shape[1]-2) # sample random token
  original_token = tokenizer.decode(t["input_ids"][0, random_token_idx]) # get the original token
  t["input_ids"][0, random_token_idx] = tokenizer.mask_token_id # mask the token
  return t, random_token_idx, original_token

# define function to get predictions for masked token
def get_predictions(tokenized, random_token_idx):
  bert_out = bert(**tokenized)
  top5 = torch.argsort(bert_out["prediction_logits"], dim=2, descending=True)[:,:,:5] # lets take top5 to answer both 1 and 2
  pred_for_mask = top5[0, random_token_idx] # from first input text, select the predictions for masked token
  return [tokenizer.decode(t) for t in pred_for_mask]

# test with one example
tokenized, random_token_idx, original_token = tokenize_and_mask_random(dataset[0]["text"])
pred_for_mask = get_predictions(tokenized, random_token_idx)

print(dataset[0]["text"])
print(tokenizer.decode(tokenized["input_ids"][0,:]))
print("Original token:", original_token)
print("Predictions:", pred_for_mask)


The film as entertainment is very good and Jimmy Stewart is excellent as Chip Hardesty, with well done co-starring turns by Vera Miles and Murray Hamilton. But the film, directed
[CLS] The film as [MASK] is very good and Jimmy Stewart is excellent as Chip Hardesty, with well done co - starring turns by Vera Miles and Murray Hamilton. But the film, directed [SEP]
Original token: entertainment
Predictions: ['well', 'Chip', 'Rocky', 'it', 'such']


In [13]:
# run all
# Note: not optimized for speed!

top1_acc = 0
top5_acc = 0

for i,review in enumerate(dataset):
  tokenized, random_token_idx, original_token = tokenize_and_mask_random(review["text"])
  pred_for_mask = get_predictions(tokenized, random_token_idx)
  if pred_for_mask[0]==original_token:
    top1_acc+=1
  if original_token in pred_for_mask:
    top5_acc+=1
  if i < 5: # print first 5 examples for visualization
    print(i)
    print(tokenizer.decode(tokenized["input_ids"][0]))
    print(original_token)
    print(pred_for_mask)
    print()

print("Top-1 accuracy:", top1_acc/len(dataset)*100)
print("Top-5 accuracy:", top5_acc/len(dataset)*100)

0
[CLS] The film as entertainment is very good and Jimmy Stewart is excellent as Chip Hardesty, with well done co - starring turns by Vera Miles and Murray Hamilton. But the [MASK], directed [SEP]
film
['film', 'movie', 'story', 'song', 'picture']

1
[CLS] If you want just about everything you want to know about WWII from multiple perspectives [MASK] this DVD delivers, you WILL learn new things guaranteed, so much so that you won't [SEP]
,
['as', 'that', 'and', 'like', 'before']

2
[CLS] A plot that fizzled and reeked of irreconcilable [MASK] in opinions constituted a judgmental havoc with one side pro - life and the other a destroyer of a demon's seed. The horror [SEP]
differences
['differences', 'conflict', 'conflicts', 'change', 'similarities']

3
[CLS] I just couldn't stop laughing!! This movie is incredibly funny and stupid! But, never [MASK] that, it is very entertaining! In this film, you don't need to pay attention to [SEP]
mind
['mind', 'forget', 'doubt', 'mention', 'say']

4


In [14]:
# data for human experiment

targets = []
for i, review in enumerate(dataset):
  tokenized, random_token_idx, original_token = tokenize_and_mask_random(review["text"])
  targets.append(original_token)
  print(i, tokenizer.decode(tokenized["input_ids"][0]))
  if i>=10: # skip rest
    break

0 [CLS] The film as entertainment is very good and Jimmy Stewart is excellent as Chip Hardesty, with well done [MASK] - starring turns by Vera Miles and Murray Hamilton. But the film, directed [SEP]
1 [CLS] If you want just about everything you want to know about WWII from multiple perspectives, this DVD delivers, you WILL learn new things guaranteed, so much [MASK] that you won't [SEP]
2 [CLS] [MASK] plot that fizzled and reeked of irreconcilable differences in opinions constituted a judgmental havoc with one side pro - life and the other a destroyer of a demon's seed. The horror [SEP]
3 [CLS] I just couldn't stop laughing!! This movie is incredibly funny and stupid! But, never mind that, it is very entertaining! In this film, you don't need [MASK] pay attention to [SEP]
4 [CLS] First, nobody can understand why this movie is rated so poorly. Not only is this the first [MASK] horrific movie since a very long time for me who am pretty [SEP]
5 [CLS] I wanted to like this film, yes its a 

In [15]:
for i, label in enumerate(targets):
  print(i, label)

0 co
1 so
2 A
3 to
4 real
5 If
6 '
7 screen
8 D
9 ##ST
10 ##bs
